In [1]:
import torch
import transformer_lens
from transformer_lens import HookedTransformer

from IPython.display import display
import circuitsvis as cv

from model import SparseAutoencoder
from config import SAEConfig



/Users/slava/safety/mats_app_nanda/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = 'cpu' # keep everything on cpu for now
checkpoints_path = "/Users/slava/fun/pos_sae/converted_checkpoints" # TODO: move checkpoints to model hub.

In [3]:
# load model
model = HookedTransformer.from_pretrained('gpt2-small', device=device)

# Load the SAEs
saes = [] # one for each layer
for layer in range(model.cfg.n_layers):
    path = f"{checkpoints_path}/final_sparse_autoencoder_gpt2-small_blocks.{layer}.hook_resid_pre_24576"
    sae = SparseAutoencoder.load_from_pretrained(path, silent=True)
    saes.append(sae)

Loaded pretrained model gpt2-small into HookedTransformer


In [4]:
### For use with activation patching ###
# text = "the team traveled by" # bus
# text = "the team succeeded by" # working together
###

In [5]:
text = "They raised awareness for the cause by"

tokens = model.tokenizer.encode(text, return_tensors="pt")
tokens = torch.cat([torch.tensor([[model.tokenizer.bos_token_id]]), tokens], dim=1) # prepend bos

logits, cache = model.run_with_cache(tokens)

In [6]:
target_feature_layer = 11
target_feature_id = 23531

# get feature activations
target_f_sae = saes[target_feature_layer]

resid_in = cache["resid_post", target_feature_layer][0]
print('resid in shape', resid_in.shape)

_, feature_acts, _, _, _ = target_f_sae(resid_in)
print('feature_acts shape', feature_acts.shape)

target_f_acts = feature_acts[:, target_feature_id]
print(target_f_acts)

resid in shape torch.Size([8, 768])
feature_acts shape torch.Size([8, 24576])
tensor([ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000, 70.1081],
       grad_fn=<SelectBackward0>)


In [19]:
# display(cv.attention.attention_patterns(
#     tokens=model.to_str_tokens(tokens),
#     attention=cache['pattern', 11][0],
#     attention_head_names=[f"L0H{i}" for i in range(12)],
# ))

In [18]:
# display(cv.attention.attention_patterns(
#     tokens=model.to_str_tokens(tokens),
#     attention=cache['pattern', 10][0],
#     attention_head_names=[f"L0H{i}" for i in range(12)],
# ))